Moving Object Speeds
====

It would be useful to get a sense of how fast various types of objects are going to be moving, to check what will leave a trail vs what will only be detected in multiple exposures. This will all be fairly obvious stuff, but collected so I have it all written down in one place.

Potential Objects:
- Low earth orbit
- Geostationary
- Main Belt Asteroids
- ~Neptune

In [20]:
import astropy.units as u
import astropy.constants as c

LEO
--
Orbit is ~90 minutes. Assuming it's directly overhead (will be reduced for more distant objects, and when some velocity is projected along the line of sight).

In [33]:
leo_altitude = 400 * u.km
leo_orbit_period = 90 * u.min
leo_phys_speed = 2*pi*(leo_altitude + c.R_earth) / leo_orbit_period
print "Physical speed: ", leo_phys_speed.to(u.km/u.s)
leo_app_speed = (leo_phys_speed * (15 * u.s) / leo_altitude * u.rad).to(u.deg)
print "LEO: ", leo_app_speed

Physical speed:  7.88671935653 km / s
LEO:  16.94534 deg


Just to check that the reflex motion is not significant,

In [34]:
(2*pi*c.R_earth / (24*u.hr)).to(u.km/u.s)

<Quantity 0.46383113891658767 km / s>

GEO
---
Orbit is 24 hours.

In [35]:
geo_altitude = 35e3 * u.km
geo_orbit_period = 24 * u.hr
geo_phys_speed = 2*pi*(geo_altitude + c.R_earth) / geo_orbit_period
print "Physical speed: ", geo_phys_speed.to(u.km/u.s)
geo_app_speed = (geo_phys_speed * (15 * u.s) / geo_altitude * u.rad).to(u.arcmin)
print "GEO: ", geo_app_speed

Physical speed:  3.00910296474 km / s
GEO:  4.43337171429 arcmin


That was actually a really dumb calculation. Geostationary objects are stationary on the sky, therefore they move one minute per minute, or 15 arcseconds per second. So really the arcs will be:

In [39]:
geo_app_speed = 15*u.s * (15 * u.arcmin/u.min)
print geo_app_speed.to(u.arcmin)

3.75 arcmin


Main Belt Asteroids
---

Most of the motion here is going to come from the Earth reflex motion.

In [57]:
#time_baseline = 15 * u.s

mba_inner_radius = 2.06 * u.au
mba_outer_radius = 3.27 * u.au
mba_inner_P = (mba_inner_radius / u.au)**(3/2.0) * u.yr
mba_outer_P = (mba_outer_radius / u.au)**(3/2.0) * u.yr
print "Inner, outer periods: ", mba_inner_P, mba_outer_P

Inner, outer periods:  2.95665621945 yr 5.91318721165 yr


At opposition:

In [58]:
solar_reflex = 2*pi*u.au / u.yr
mba_inner_v = 2*pi*mba_inner_radius/mba_inner_P
mba_outer_v = 2*pi*mba_outer_radius/mba_outer_P

Over a 15 second snap:

In [73]:
time_baseline = 15*u.s
mba_inner_ang = (mba_inner_v - solar_reflex) * time_baseline / (mba_inner_radius - 1*u.au) * u.rad
mba_outer_ang = (mba_outer_v - solar_reflex) * time_baseline / (mba_outer_radius - 1*u.au) * u.rad
print "Inner Edge: {0:.2f}".format(mba_inner_ang.to(u.arcsec))
print "Outer Edge: {0:.2f}".format(mba_outer_ang.to(u.arcsec))

Inner Edge: -0.18 arcsec
Outer Edge: -0.12 arcsec


Between three day visits:

In [75]:
time_baseline = 3*u.day
mba_inner_ang = (mba_inner_v - solar_reflex) * time_baseline / (mba_inner_radius - 1*u.au) * u.rad
mba_outer_ang = (mba_outer_v - solar_reflex) * time_baseline / (mba_outer_radius - 1*u.au) * u.rad
print "Inner Edge: {0:.2f}".format(mba_inner_ang.to(u.deg))
print "Outer Edge: {0:.2f}".format(mba_outer_ang.to(u.deg))

Inner Edge: -0.85 deg
Outer Edge: -0.58 deg


Let's just generalize this:

In [77]:
def ss_object_angular_speed(heliocentric_dist):
    solar_reflex = 2*pi*u.au / u.yr
    obj_P = (heliocentric_dist / u.au)**(3/2.0) * u.yr
    obj_vel = 2*pi*heliocentric_dist/obj_P
    ang_vel = (obj_vel - solar_reflex) / (heliocentric_dist - 1*u.au) * u.rad
    return ang_vel
    

In [88]:
neptune_angvel = ss_object_angular_speed(35*u.au)
print "Neptune, snap: {0:.3f}".format((neptune_angvel*(15 *u.s)).to(u.arcsec))
print "Neptune, visits: {0:.3f}".format((neptune_angvel*(3*u.day)).to(u.arcmin))

Neptune, snap: -0.015 arcsec
Neptune, visits: -4.336 arcmin


Synthesis
---
This shows that:
- LEO objects will cross the entire focal plane, or much of it.
- GEO objects will appear as small tracks, easily recongizable
- Asteroids will clearly be the hard part, since they won't appear as tracks, but will have high enough motion to move significantly between visits.
- Distant objects will have short motions between visits, should be easily linked.